## Import Modules

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, f1_score
from sklearn.feature_selection import RFE

from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
import xgboost as xgb

In [2]:
pd.set_option('display.max_columns', None)

## Import Data

In [3]:
raw_data = pd.read_csv(r"C:\Users\Abraham Audu\Documents\Py-Self-Learn\credit_card_default_prediction\data\UCI_Credit_Card.csv")

In [4]:
raw_data.head()

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT1,BILL_AMT2,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default.payment.next.month
0,1,20000.0,2,2,1,24,2,2,-1,-1,-2,-2,3913.0,3102.0,689.0,0.0,0.0,0.0,0.0,689.0,0.0,0.0,0.0,0.0,1
1,2,120000.0,2,2,2,26,-1,2,0,0,0,2,2682.0,1725.0,2682.0,3272.0,3455.0,3261.0,0.0,1000.0,1000.0,1000.0,0.0,2000.0,1
2,3,90000.0,2,2,2,34,0,0,0,0,0,0,29239.0,14027.0,13559.0,14331.0,14948.0,15549.0,1518.0,1500.0,1000.0,1000.0,1000.0,5000.0,0
3,4,50000.0,2,2,1,37,0,0,0,0,0,0,46990.0,48233.0,49291.0,28314.0,28959.0,29547.0,2000.0,2019.0,1200.0,1100.0,1069.0,1000.0,0
4,5,50000.0,1,2,1,57,-1,0,-1,0,0,0,8617.0,5670.0,35835.0,20940.0,19146.0,19131.0,2000.0,36681.0,10000.0,9000.0,689.0,679.0,0


## EDA

In [5]:
# Check for unique value count
raw_data.nunique()

ID                            30000
LIMIT_BAL                        81
SEX                               2
EDUCATION                         7
MARRIAGE                          4
AGE                              56
PAY_0                            11
PAY_2                            11
PAY_3                            11
PAY_4                            11
PAY_5                            10
PAY_6                            10
BILL_AMT1                     22723
BILL_AMT2                     22346
BILL_AMT3                     22026
BILL_AMT4                     21548
BILL_AMT5                     21010
BILL_AMT6                     20604
PAY_AMT1                       7943
PAY_AMT2                       7899
PAY_AMT3                       7518
PAY_AMT4                       6937
PAY_AMT5                       6897
PAY_AMT6                       6939
default.payment.next.month        2
dtype: int64

In [6]:
# Check for missing values
raw_data.isnull().sum()

ID                            0
LIMIT_BAL                     0
SEX                           0
EDUCATION                     0
MARRIAGE                      0
AGE                           0
PAY_0                         0
PAY_2                         0
PAY_3                         0
PAY_4                         0
PAY_5                         0
PAY_6                         0
BILL_AMT1                     0
BILL_AMT2                     0
BILL_AMT3                     0
BILL_AMT4                     0
BILL_AMT5                     0
BILL_AMT6                     0
PAY_AMT1                      0
PAY_AMT2                      0
PAY_AMT3                      0
PAY_AMT4                      0
PAY_AMT5                      0
PAY_AMT6                      0
default.payment.next.month    0
dtype: int64

## Preprocessing & Feature Engineering

In [7]:
data = raw_data.copy()
data = data.drop('ID', axis=1)

In [8]:
# Get categorical and numerical features
categorical_features = [feature for feature in data.columns if data[feature].nunique() < 20 and feature != 'default.payment.next.month']
continuous_features = [feature for feature in data.columns if feature not in categorical_features and feature != 'default.payment.next.month']

Encode categorical features

In [9]:
data_temp = data.copy()
data_to_encode = data_temp[categorical_features]

# Initialize and fit encoder 
encoder = OneHotEncoder(sparse_output=False, drop='first')
encoder.fit(data_to_encode)

# Transform data
encoded_columns = encoder.transform(data_to_encode)
encoded_df = pd.DataFrame(encoded_columns, 
                          columns=encoder.get_feature_names_out(categorical_features))

data_temp.drop(categorical_features, axis=1, inplace=True)
data_encoded = pd.concat([data_temp, encoded_df], axis=1)

In [10]:
data_encoded.columns

Index(['LIMIT_BAL', 'AGE', 'BILL_AMT1', 'BILL_AMT2', 'BILL_AMT3', 'BILL_AMT4',
       'BILL_AMT5', 'BILL_AMT6', 'PAY_AMT1', 'PAY_AMT2', 'PAY_AMT3',
       'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6', 'default.payment.next.month',
       'SEX_2', 'EDUCATION_1', 'EDUCATION_2', 'EDUCATION_3', 'EDUCATION_4',
       'EDUCATION_5', 'EDUCATION_6', 'MARRIAGE_1', 'MARRIAGE_2', 'MARRIAGE_3',
       'PAY_0_-1', 'PAY_0_0', 'PAY_0_1', 'PAY_0_2', 'PAY_0_3', 'PAY_0_4',
       'PAY_0_5', 'PAY_0_6', 'PAY_0_7', 'PAY_0_8', 'PAY_2_-1', 'PAY_2_0',
       'PAY_2_1', 'PAY_2_2', 'PAY_2_3', 'PAY_2_4', 'PAY_2_5', 'PAY_2_6',
       'PAY_2_7', 'PAY_2_8', 'PAY_3_-1', 'PAY_3_0', 'PAY_3_1', 'PAY_3_2',
       'PAY_3_3', 'PAY_3_4', 'PAY_3_5', 'PAY_3_6', 'PAY_3_7', 'PAY_3_8',
       'PAY_4_-1', 'PAY_4_0', 'PAY_4_1', 'PAY_4_2', 'PAY_4_3', 'PAY_4_4',
       'PAY_4_5', 'PAY_4_6', 'PAY_4_7', 'PAY_4_8', 'PAY_5_-1', 'PAY_5_0',
       'PAY_5_2', 'PAY_5_3', 'PAY_5_4', 'PAY_5_5', 'PAY_5_6', 'PAY_5_7',
       'PAY_5_8', 'PAY_6_-1', 'PAY_

Undersample the data

In [11]:
# Undersample any target label with count > mean count of the dataset

def undersample_by_value_counts(data, label_column):
    value_counts = data[label_column].value_counts()
    mean_count = value_counts.mean()

    undersampled_data = pd.DataFrame(columns=data.columns)

    for value, count in value_counts.items():
        if count > mean_count:
            undersampled_count = int((count / value_counts.sum()) * (mean_count/2))
            subset = data[data[label_column] == value].sample(n=undersampled_count, random_state=42)
            undersampled_data = pd.concat([undersampled_data, subset], ignore_index=True)
        else:
            subset = data[data[label_column] == value]
            undersampled_data = pd.concat([undersampled_data, subset], ignore_index=True)

    # Randomize the undersampled data
    randomized_data = undersampled_data.sample(frac=1, random_state=42)

    return randomized_data.astype('int64')

In [12]:
undersampled_data = undersample_by_value_counts(data, 'default.payment.next.month')

In [13]:
undersampled_data_encoded = undersample_by_value_counts(data_encoded, 'default.payment.next.month')

In [14]:
undersampled_data['default.payment.next.month'].value_counts()

1    6636
0    5841
Name: default.payment.next.month, dtype: int64

## Model Experiments

### Train-Test Split

In [30]:
# select data to split
data_used = undersampled_data_encoded

X = data_used.drop('default.payment.next.month', axis=1) # Inputs
y = data_used['default.payment.next.month'] # Target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, 
                                                    random_state=42, 
                                                    stratify=y,
                                                    shuffle=True)

### Scale the Inputs

In [31]:
scaler = StandardScaler().fit(X_train)

X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [32]:
X_train_scaled.shape

(9981, 82)

### Feature Selection - Recursive Feature Elimination

Setup Baseline Model - Gradient Boosting Classifier

In [33]:
# Classification Report for default classes
def print_f1_scores(y_true, y_pred):
    report = classification_report(y_true, y_pred, output_dict=True)

    for class_label, metric in report.items():
        try:
            f1_score_float = metric['f1-score']
            print(f"Class {class_label}: F1-score = {round(f1_score_float, 3)}")
        except:
            pass

In [34]:
# Initialize classifier
gbc = GradientBoostingClassifier(random_state=42)

# Train classifier using all features
gbc.fit(X_train_scaled, y_train)

# Make predictions
preds = gbc.predict(X_test_scaled)

# Evaluate model (F1 Score)
print_f1_scores(y_test, preds)

Class 0: F1-score = 0.7
Class 1: F1-score = 0.701
Class macro avg: F1-score = 0.701
Class weighted avg: F1-score = 0.701


Setup RFE Feature Selector

In [35]:
# f1_score_list_weighted = []
# f1_score_list_macro = []

# for k in range(5,25,3):
#     RFE_selector = RFE(estimator=gbc, n_features_to_select=k, step=5)
#     RFE_selector.fit(X_train_scaled, y_train)

#     sel_X_train = RFE_selector.transform(X_train_scaled)
#     sel_X_test = RFE_selector.transform(X_test_scaled)

#     gbc.fit(sel_X_train, y_train)
#     RFE_preds = gbc.predict(sel_X_test)

#     f1_score_rfe_weighted = round(f1_score(y_test, RFE_preds, average='weighted'), 3)
#     f1_score_rfe_macro = round(f1_score(y_test, RFE_preds, average='macro'), 3)
#     f1_score_list_weighted.append(f1_score_rfe_weighted)
#     f1_score_list_macro.append(f1_score_rfe_macro)

#     print(f"Class scores for {k} features model:")
#     print_f1_scores(y_test, RFE_preds)
#     print('##----------------------------------------##\n')

In [36]:
list(range(5,25,3))

[5, 8, 11, 14, 17, 20, 23]

In [37]:
# models = {
#     LogisticRegression(): 'Logistic Regression',
#     SVC(): 'C-Support Vector Classification',
#     MLPClassifier(): 'Neural Network (Multi-layer Perceptron classifier)',
#     RandomForestClassifier(): 'Random Forest',
#     DecisionTreeClassifier(): 'Decision Tree',
#     xgb.XGBClassifier(): 'XGB Classifier'
#     }

# for model in models.keys():
#     model.fit(X_train_scaled, y_train)


In [38]:
# for model, name in models.items():
#     y_true = y_test.copy()
#     y_pred = model.predict(X_test_scaled)
    
#     accuracy = model.score(X_test_scaled, y_test)
#     F1_SCORE = f1_score(y_true, y_pred, average='macro')
    
#     print(f"{name}: \naccuracy --> {accuracy}\nf1_score --> {F1_SCORE}\n")

In [50]:
model_params = {
    'Logistic Regression': {
        'model': LogisticRegression(),
        'params': {
            'C': list(range(1,30,9)),
            'solver': ['liblinear', 'newton-cg']
        }
    },
    
    # 'C-Support Vector Classification': {
    #     'model': SVC(),
    #     'params': {
    #         'C': list(range(1,20,10)),
    #         'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
    #         'gamma': ['scale', 'auto']
    #     }
    # },
    
    # 'Neural Network (Multi-layer Perceptron classifier)': {
    #     'model': MLPClassifier(),
    #     'params': {
    #         'hidden_layer_sizes': list(range(100,450,200)),
    #         'activation': ['identity', 'logistic', 'tanh', 'relu'],
    #         'solver': ['lbfgs', 'sgd', 'adam'],
    #         # 'max_iter': list(range(200,450,100))
    #     }
    # },
    
    # 'Random Forest': {
    #     'model': RandomForestClassifier(),
    #     'params': {
    #         'criterion': ['gini', 'entropy', 'log_loss'],
    #         'max_features': ["sqrt", "log2", None]
    #     }
    # }
}

In [51]:
scores = []

for model_name, mp in model_params.items():
    run = GridSearchCV(mp['model'], mp['params'], return_train_score=False, n_jobs=-1, scoring = 'f1', verbose = 1, cv=2)
    run.fit(X_train, y_train)
    scores.append({
        'model': model_name,
        'best_score': run.best_score_,
        'best_params': run.best_params_
    })

Fitting 2 folds for each of 8 candidates, totalling 16 fits


In [52]:
scores

[{'model': 'Logistic Regression',
  'best_score': 0.6824424114473375,
  'best_params': {'C': 19, 'solver': 'liblinear'}}]

In [46]:
import os
os.getcwd()

'c:\\Users\\Abraham Audu\\Documents\\Py-Self-Learn\\credit_card_default_prediction\\experiments'